## Atributos generados

In [ ]:
execID=23
algorithm = "MosaicoDosUnidades"
version= "1.0"

## Parámetros comunes

In [ ]:
products = ['LS5_TM_LEDAPS','LS7_ETM_LEDAPS' ] #Productos sobre los que se hará la consulta (unidades de almacenamiento)
bands=["blue","green","red","nir", "swir1","swir2"] #arreglo de bandas 
time_ranges = [("2015-04-01", "2015-06-30")] #Una lista de tuplas, cada tupla representa un periodo
#área sobre la cual se hará la consulta:
min_long = -71
min_lat = -5
max_long = -69
max_lat = -2

## Parámetros específicos del algoritmo

In [ ]:
normalized=True
minValid=1;

# Librerías

In [ ]:
import datacube
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
import os
import re
import xarray as xr
import numpy as np
import gdal


# Consulta

Consulta sobre las diferentes unidades y aplica la máscara de nubes adecuada. 

In [ ]:
nodata=-9999
#Definir las funciones necesatias para el algoritmo
def isin(element, test_elements, assume_unique=False, invert=False):
    "definiendo la función isin de numpy para la versión anterior a la 1.13, en la que no existe"
    element = np.asarray(element)
    return np.in1d(element, test_elements, assume_unique=assume_unique,
                invert=invert).reshape(element.shape)

### Máscara de nubes

Con el nuevo formato, los valores de `pixel_qa` dependen del producto. Para crear la máscara de nubes, se determinan los valores válidos para el producto actual y se usa la banda `pixel_qa` para generar un arreglo de datos booleanos: Para cada posición, si el valor de pixel_qa está en la lista de valores válidos será `True`, en caso contrario será `False`.

In [ ]:
kwargs={}
dc = datacube.Datacube(app="{}_{}_{}".format(algorithm,version,execID))
for product in products:
    i=0
    validValues=set()
    if product=="LS5_TM_LEDAPS":
        validValues=[66,68,130,132]
    elif product == "LS7_ETM_LEDAPS":
        validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
    for tr in time_ranges:
        _data = dc.load(product=product, longitude=(min_long, max_long), latitude=(min_lat, max_lat), time=tr)
        if len(_data.data_vars)==0:
            break
        cloud_mask=isin(_data["pixel_qa"].values, validValues)
        for band in bands:
            _data[band].values=np.where(np.logical_and(_data.data_vars[band]!=nodata,cloud_mask),_data.data_vars[band], np.nan)
        _undesired=list(set(_data.keys())-set(bands+['latitude','longitude','time']))
        _data=_data.drop(_undesired)
            
        if "xarr"+str(i) in kwargs:
            kwargs["xarr"+str(i)]=xr.concat([kwargs["xarr"+str(i)],_data.copy(deep=True)], 'time')
        else:
            kwargs["xarr"+str(i)]=_data
    i+=1
del _data

In [ ]:
_undesired

In [ ]:
#El algoritmo recibe los productos como xarrays en variablles llamadas xarr0, xarr1, xarr2... 
xarr0=kwargs["xarr0"]
del kwargs

In [ ]:
xarr0

# Procesamiento

El algoritmo debe ser auto contenido. Puede importar y usar las librerías disponibles en CDCol, por ejemplo: 

- scikit-learn
- numpy
- xarray
- pycurl
- pandas
- nltk

## Compuesto temporal de medianas

In [ ]:
medians={} 
for band in bands:
    datos=xarr0[band].values
    allNan=~np.isnan(datos) #Una mascara que indica qué datos son o no nan. 
    if normalized: #Normalizar, si es necesario.
        #Para cada momento en el tiempo obtener el promedio y la desviación estándar de los valores de reflectancia
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        # usar ((x-x̄)/st) para llevar la distribución a media 0 y desviación estándar 1, 
        # y luego hacer un cambio de espacio para la nueva desviación y media. 
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    #Calcular la mediana en la dimensión de tiempo 
    medians[band]=np.nanmedian(datos,0) 
    #Eliminar los valores que no cumplen con el número mínimo de pixeles válidos dado. 
    medians[band][np.sum(allNan,0)<minValid]=np.nan
del datos


In [ ]:
_coords=xarr0.coords
_crs=xarr0.crs
del xarr0

## Preparar la salida
La salida de los algoritmos puede expresarse como: 
- un xarray llamado output (que debe incluir entre sus atributos el crs del sistema de coordenadas)
- un diccionario con varios xarray llamado `outputs`
- Texto, en una variable llamada `outputtxt`

In [ ]:
import xarray as xr
ncoords=[]
xdims =[]
xcords={}
for x in _coords:
    if(x!='time'):
        ncoords.append( ( x, _coords[x]) )
        xdims.append(x)
        xcords[x]=_coords[x]
variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords)
             for k, v in medians.items()}
output=xr.Dataset(variables, attrs={'crs':_crs})

for x in output.coords:
    _coords[x].attrs["units"]=_coords[x].units

# Guardar la salida
La tarea genérica se encarga de generar los archivos de salida en la carpeta adecuada. 

__Nota__: A diferencia de la tarea genérica, que maneja los 3 tipos de salida descritos en la sección anterior, este cuaderno sólo guarda la salida definida en output

In [ ]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
print "{}_{}_{}.nc".format(algorithm,version,execID)
nco=netcdf_writer.create_netcdf("{}_{}_{}.nc".format(algorithm,version,execID))
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, _coords[x].values, _coords[x].units)
netcdf_writer.create_grid_mapping_variable(nco, _crs)
for band in bands:
    medians[band][np.isnan(medians[band])]=nodata
    var= netcdf_writer.create_variable(nco, band, Variable(np.dtype(np.int32), None, ('latitude', 'longitude'), None) ,set_crs=True)
    var[:] = netcdf_writer.netcdfy_data(medians[band])
nco.close()